In [ ]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px
from rouge_score import rouge_scorer

# **I - Chargement des données**

In [2]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

# Ajout de nouvelles colonnes
### Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

### Nombre de charactères par contribution et par extractions
df["nb_char_contrib"] = df["contribution"].apply(len)
df["nb_char_extraction"] = df["ideas_text"].apply(len)

### Hallucinations et Idées_non_ind en catégories
df['hal_cat'] = df['Hallucinations'].astype("category")
df["idees_cat"] = df["Idées_non_ind"].astype("category")

### Catégories de longueur
length_bins = [0, 250, 500, 2000, float("inf")]
length_labels = ["Très courte (0-250)", "Courte (251-500)", "Moyenne (501-2000)", "Longue (2000+)"]
df["contrib_cat"] = pd.cut(df["nb_char_contrib"], bins=length_bins, labels=length_labels)

# **III - Analyse de la métrique**

les métriques ROUGE (Recall-Oriented Understudy for Gisting Evaluation ) est un ensemble de métriques qui sont généralement utilisées pour évaluier la qualité des résumés proposées par un LLM. Ces différentes métriques repose dans notre cas sur les similarités entre les mots de l'extraction du LLM et l'idée de base. On peut définir 3 catégories de métriques ROUGE :
- N-grammes : compare les groupes de N mots consécutifs entre l'idée originel et l'extraction
- L ( Longest Common Subsequence) : mesure la similarité en séquence plutot qu'en N-gramm
- S/SU : Statistique de co-occurence reposant sur les bigrammes de saut (paire de mots quelconque dans leur ordre de phrase): bigramme sauté et les unigrammes

Celles-ci sont comprises entre 0 et 1 : plus la métrique est proche de 1 et plus l'extraction est de bonne qualité. Inversement, plus la métrique est proche de 0 et plus elle est de mauvaise qualité.

Dans notre cas, on cherche à évaluer la qualité des idées extraites plus qu'une similarité mot à mot. La métrique la plmus adapté semble donc etre la métrique LCS. 
Pour implémenter cette métrique avec Python, on emploiera le package [rouge-score](https://pypi.org/project/rouge-score/). L'exemple de base sur ce site calcule la métrique rouge &-gramm et L, ce que l'on va calculer puis implémenter dans le jeud e donnée df 

In [3]:
# On crée les vars pour la métrique ROUGE
df["1gramm"],df["L"] = 0,0

# Les 2 métriques à évaluer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

for i in range(np.shape(df)[0]):
    # try pour que ca passe avec les parse error
    try:
        ref = str(df.loc[i, 'ideas_text'])
        pred = str(df.loc[i, 'contribution'])
        
        scores = scorer.score(ref, pred)
        
        df.loc[i, '1gramm'] = scores['rouge1'].fmeasure
        df.loc[i, 'L'] = scores['rougeL'].fmeasure
    
    # 0 si ca ne passe pas
    except Exception as e:
        print(f"Erreur à la ligne {i} : {e}")
        df.loc[i, '1gramm'] = 0
        df.loc[i, 'L'] = 0

/var/folders/9x/5wd13t7x7377s56tp32vtnnr0000gn/T/ipykernel_31010/1231880777.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0014492753623188406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, '1gramm'] = scores['rouge1'].fmeasure
/var/folders/9x/5wd13t7x7377s56tp32vtnnr0000gn/T/ipykernel_31010/1231880777.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0014492753623188406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'L'] = scores['rougeL'].fmeasure


In [4]:
# Corrélation de Pearson entre score humain et score ROUGE &gramm/L
correlation_1gramm = df["score_humain"].corr(df["1gramm"])
print(f"Corrélation entre le score humain moyen et le score ROUGE 1gramm : {correlation_1gramm:.2f}")

correlation_L = df["score_humain"].corr(df["L"])
print(f"Corrélation entre le score humain moyen et le score ROUGE L : {correlation_L:.2f}")

Corrélation entre le score humain moyen et le score ROUGE 1gramm : 0.79
Corrélation entre le score humain moyen et le score ROUGE L : 0.78


Les corrélation entre l'évaluation humaine et celles des métriques ROUGE est plus grande que celle de QualIT (0.71)

In [5]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="1gramm", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["1gramm"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique ROUGE 1-gramm = %{x}<br>Score humain = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    # title="<b>Score humain vs Métrique ROUGE 1-gramm</b>",
    xaxis_title="1-gram ROUGE metric", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()

In [6]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="L", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["L"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique ROUGE L = %{x}<br>Score humain = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    # title="<b>Score humain vs Métrique ROUGE L</b>",
    xaxis_title="L ROUGE metric", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()